In [ ]:
! pip install --upgrade imgaug

# Importation des modules utiles

In [ ]:
import numpy as np
import pandas as pd
import os
import albumentations as albu
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import colorama
from colorama import Fore
from imgaug.augmentables.segmaps import SegmentationMapOnImage

from tensorflow.keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence

### Import des scripts de fonctions utiles

In [ ]:
from clouds_graph_functions import visualize_image_mask_prediction
from clouds_utilities_functions import np_resize, build_masks
from clouds_utilities_functions import dice_coef, dice_loss, bce_dice_loss, dice_coef_class

# Définition du chemin des données

In [ ]:
NUAGES_PATH = '/kaggle/input/understanding_cloud_organization/'

NUAGES_TRAIN_PATH = NUAGES_PATH + 'train_images/'

# Chargement du DataFrame et modification pour le DataGenerator

In [ ]:
# Chargement du jeu de données d'entrainement
train_df = pd.read_csv(NUAGES_PATH + 'train.csv')

In [ ]:
train_df['ImageId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

In [ ]:
print(train_df.shape)
train_df.head()

In [ ]:
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)

In [ ]:
print(mask_count_df.shape)
mask_count_df.head()

## Parametres generaux

In [ ]:
BATCH_SIZE = 6
HEIGHT = 320
WIDTH = 480
CHANNELS = 3 # toujours garder 3 channels car les modèles d'initialisation des poids ont été entrainé sur des images couleurs avec 3 canaux
COLOR_MODE=True # True pour images couleurs et False pour images en noir et blanc
NB_CLASSES = 4

# Data Generator pour image et masque

In [ ]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path=NUAGES_TRAIN_PATH, batch_size=BATCH_SIZE, dim=(1400, 2100),
                 n_channels=CHANNELS, color_mode=True, reshape=None, augment=False,
                 n_classes=NB_CLASSES, random_state=222, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.reshape = reshape
        self.n_channels = n_channels
        self.color_mode = color_mode
        self.augment = augment
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()
        np.random.seed(self.random_state)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            
            if self.augment:
                X, y = self.__augment_batch(X, y)
            
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)          

    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        if self.reshape is None:
            X = np.empty((self.batch_size, *self.dim, self.n_channels))
        else:
            X = np.empty((self.batch_size, *self.reshape, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}{im_name}"
            
            if self.color_mode == True:
                img = self.__load_rgb(img_path)
                
                if self.reshape is not None:
                    img = np_resize(img, self.reshape)
            
            else:
                img = self.__load_grayscale(img_path)
                
                if self.reshape is not None:
                    img = np_resize(img, self.reshape)
                
                img = np.dstack((img,)*self.n_channels) # stack the black and white image in n channels
            
            # Store samples
            X[i,] = img

        return X

    def __generate_y(self, list_IDs_batch):
        if self.reshape is None:
            y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        else:
            y = np.empty((self.batch_size, *self.reshape, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            
            if self.reshape is not None:
                masks = build_masks(rles, input_shape=self.dim, reshape=self.reshape)
            else:
                masks = build_masks(rles, input_shape=self.dim)
            
            y[i, ] = masks

        return y
 
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)
        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.
        return img
    
    def __random_transform(self, img, masks):
        #augmentation random d'images
        composition = albu.Compose([albu.HorizontalFlip(),
                                    albu.VerticalFlip(),
                                    albu.Rotate(limit=20),
                                    albu.GridDistortion(),
                                    albu.ShiftScaleRotate(rotate_limit=45, shift_limit=0.15, scale_limit=0.15)])
        
        composed = composition(image=img, mask=masks)
        aug_img = composed['image']
        aug_masks = composed['mask']
        
        return aug_img, aug_masks  
    
    def __augment_batch(self, img_batch, masks_batch):
        # generation du batch d'augmentation de l'image et de son masque associé
        for i in range(img_batch.shape[0]):
            img_batch[i, ], masks_batch[i, ] = self.__random_transform(img_batch[i, ], masks_batch[i, ])
        
        return img_batch, masks_batch

# Training

## Séparation des données d'entrainement et de validation (par ID unique)

In [ ]:
train_idx, val_idx = train_test_split(mask_count_df.index, random_state=69, test_size=0.2)

## Création des instances de la class DataGenerator

In [ ]:
train_generator = DataGenerator(train_idx, 
                                df=mask_count_df,
                                target_df=train_df,
                                batch_size=BATCH_SIZE,
                                reshape=(HEIGHT, WIDTH),
                                augment=True,
                                n_channels=CHANNELS,
                                color_mode = COLOR_MODE,
                                n_classes=NB_CLASSES)

valid_generator = DataGenerator(val_idx, 
                                  df=mask_count_df,
                                  target_df=train_df,
                                  batch_size=BATCH_SIZE, 
                                  reshape=(HEIGHT, WIDTH),
                                  augment=False,
                                  n_channels=CHANNELS,
                                  color_mode = COLOR_MODE,
                                  n_classes=NB_CLASSES)

## Callbacks

In [ ]:
# ModelCheckpoint callback : pour enregistrer les poids du modele.
checkpoint = ModelCheckpoint("./model.h5",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_loss',
                               mode='min',
                               min_delta = 0.01,
                               patience=5,
                               restore_best_weights=True,
                               verbose=1)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         mode='min',
                                         episilon = 0.01,
                                         patience=3,
                                         factor=0.1,
                                         min_lr=1e-6,
                                         verbose=1)

# CSVLogger callback : pour enregistrer l'historique d'entrainement.
csv_logger = CSVLogger('./training.log')

## MODELE de segmentation

In [ ]:
#creation d'un modéle SEGNET avec un backbone VGG16 from scratch
#Step #1: Creation des class MaxPoolingWithArgmax2D ( pour l'encoder) et MaxUnpooling2D ( pour le decoder)

from keras import backend as K
from keras.layers import Layer
import tensorflow as tf


class MaxPoolingWithArgmax2D(Layer):
    def __init__(self, pool_size=(2, 2), strides=(2, 2), padding="same", **kwargs):
        super(MaxPoolingWithArgmax2D, self).__init__(**kwargs)
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides

    def call(self, inputs, **kwargs):
        padding = self.padding
        pool_size = self.pool_size
        strides = self.strides
        if K.backend() == "tensorflow":
            ksize = [1, pool_size[0], pool_size[1], 1]
            padding = padding.upper()
            strides = [1, strides[0], strides[1], 1]
            output, argmax =tf.nn.max_pool_with_argmax(
                inputs, ksize=ksize, strides=strides, padding=padding
            )
        else:
            errmsg = "{} backend is not supported for layer {}".format(
                K.backend(), type(self).__name__
            )
            raise NotImplementedError(errmsg)
        argmax = K.cast(argmax, K.floatx())
        return [output, argmax]

    def compute_output_shape(self, input_shape):
        ratio = (1, 2, 2, 1)
        output_shape = [
            dim // ratio[idx] if dim is not None else None
            for idx, dim in enumerate(input_shape)
        ]
        output_shape = tuple(output_shape)
        return [output_shape, output_shape]

    def compute_mask(self, inputs, mask=None):
        return 2 * [None]


class MaxUnpooling2D(Layer):
    def __init__(self, size=(2, 2), **kwargs):
        super(MaxUnpooling2D, self).__init__(**kwargs)
        self.size = size

    def call(self, inputs, output_shape=None):
        updates, mask = inputs[0], inputs[1]
        with tf.variable_scope(self.name):
            mask = K.cast(mask, "int32")
            input_shape = tf.shape(updates, out_type="int32")
            #  calculation new shape
            if output_shape is None:
                output_shape = (
                    input_shape[0],
                    input_shape[1] * self.size[0],
                    input_shape[2] * self.size[1],
                    input_shape[3],
                )
            self.output_shape1 = output_shape

            # calculation indices for batch, height, width and feature maps
            one_like_mask = K.ones_like(mask, dtype="int32")
            batch_shape = K.concatenate([[input_shape[0]], [1], [1], [1]], axis=0)
            batch_range = K.reshape(
                tf.range(output_shape[0], dtype="int32"), shape=batch_shape
            )
            b = one_like_mask * batch_range
            y = mask // (output_shape[2] * output_shape[3])
            x = (mask // output_shape[3]) % output_shape[2]
            feature_range = tf.range(output_shape[3], dtype="int32")
            f = one_like_mask * feature_range

            # transpose indices & reshape update values to one dimension
            updates_size = tf.size(updates)
            indices = K.transpose(K.reshape(K.stack([b, y, x, f]), [4, updates_size]))
            values = K.reshape(updates, [updates_size])
            ret = tf.scatter_nd(indices, values, output_shape)
            return ret

    def compute_output_shape(self, input_shape):
        mask_shape = input_shape[1]
        return (
            mask_shape[0],
            mask_shape[1] * self.size[0],
            mask_shape[2] * self.size[1],
            mask_shape[3],
        )

In [ ]:
#creation d'un modéle SEGNET avec un backbone VGG16
#Step #2: Creation du modéle : encoder + decoder

from keras.layers import Input
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import layers
import keras
import urllib.request


def segnet(input_shape, n_labels, kernel=3, pool_size=(2, 2), output_mode="sigmoid"):
    # encoder
    inputs = Input(shape=input_shape)

    conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation("relu")(conv_2)

    pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

    conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Activation("relu")(conv_4)

    pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

    conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Activation("relu")(conv_6)
    conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
    conv_7 = BatchNormalization()(conv_7)
    conv_7 = Activation("relu")(conv_7)

    pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

    conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Activation("relu")(conv_8)
    conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
    conv_9 = BatchNormalization()(conv_9)
    conv_9 = Activation("relu")(conv_9)
    conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
    conv_10 = BatchNormalization()(conv_10)
    conv_10 = Activation("relu")(conv_10)

    pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

    conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Activation("relu")(conv_11)
    conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
    conv_12 = BatchNormalization()(conv_12)
    conv_12 = Activation("relu")(conv_12)
    conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
    conv_13 = BatchNormalization()(conv_13)
    conv_13 = Activation("relu")(conv_13)

    pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
    print("Build enceder done..")    

    # decoder

    unpool_1 = MaxUnpooling2D(pool_size)([pool_5, mask_5])

    conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_1)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Activation("relu")(conv_14)
    conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
    conv_15 = BatchNormalization()(conv_15)
    conv_15 = Activation("relu")(conv_15)
    conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
    conv_16 = BatchNormalization()(conv_16)
    conv_16 = Activation("relu")(conv_16)

    unpool_2 = MaxUnpooling2D(pool_size)([conv_16, mask_4])

    conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_2)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
    conv_18 = BatchNormalization()(conv_18)
    conv_18 = Activation("relu")(conv_18)
    conv_19 = Convolution2D(256, (kernel, kernel), padding="same")(conv_18)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_3 = MaxUnpooling2D(pool_size)([conv_19, mask_3])

    conv_20 = Convolution2D(256, (kernel, kernel), padding="same")(unpool_3)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Convolution2D(256, (kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Activation("relu")(conv_21)
    conv_22 = Convolution2D(128, (kernel, kernel), padding="same")(conv_21)
    conv_22 = BatchNormalization()(conv_22)
    conv_22 = Activation("relu")(conv_22)

    unpool_4 = MaxUnpooling2D(pool_size)([conv_22, mask_2])

    conv_23 = Convolution2D(128, (kernel, kernel), padding="same")(unpool_4)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Convolution2D(64, (kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Activation("relu")(conv_24)

    unpool_5 = MaxUnpooling2D(pool_size)([conv_24, mask_1])

    conv_25 = Convolution2D(64, (kernel, kernel), padding="same")(unpool_5)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu")(conv_25)

    conv_26 = Convolution2D(n_labels, (1, 1), padding="valid")(conv_25)

    outputs = Activation("sigmoid")(conv_26)
    print("Build decoder done..")
    
    model = Model(inputs=inputs, outputs=outputs, name="SegNet")
    
    #chargement du modele VGG16 pré-entrainé  
    pretrained_url = "https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
    VGG_Weights_path = keras.utils.get_file(pretrained_url.split("/")[-1], pretrained_url)
    
    model.load_weights(VGG_Weights_path, by_name=True, skip_mismatch=True)
  

    return model

## Chargement des poids du modèle pré entrainé si existant

In [ ]:
input_model_path = "/kaggle/input/clouds-segmentation-segnet/model.h5"

try:
    model.load_weights(input_model_path)
    print("model.h5 pré entrainé chargé!")
except:
    print("model.h5 non pré entrainé, non chargé")
    pass

In [ ]:
input_shape=(HEIGHT,WIDTH,3)

model= segnet(input_shape, NB_CLASSES, kernel=3, pool_size=(2, 2), output_mode="sigmoid")
model.compile(optimizer="adam", loss=bce_dice_loss, metrics=[dice_coef])
model.summary()

# Entrainement

In [ ]:
EPOCHS = 10

model_info = model.fit_generator(generator=train_generator,
                                  validation_data=valid_generator,
                                  callbacks=[checkpoint, early_stopping, reduce_learning_rate, csv_logger],
                                  epochs=EPOCHS)

## Courbes des résultats de l'entrainement

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(221)
plt.plot(model_info.history['loss'])
plt.plot(model_info.history['val_loss'])
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='right')

plt.subplot(222)
plt.plot(model_info.history['dice_coef'])
plt.plot(model_info.history['val_dice_coef'])
plt.title('Model dice_coef by epoch')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='right')
plt.show()


## Chargement des poids du modèle après entrainement

In [ ]:
output_model_path = "./model.h5"

try:
    try:
        # Si entrainement du modele, les nouveaux poids sont dans output
        model.load_weights(output_model_path)
        print("model.h5 en output chargé!")
    except:
        print("model.h5 non entrainé en output, non chargé")
        try:
            # Si modele pré entrainé en input
            model.load_weights(input_model_path)
            print("model.h5 pré entrainé chargé!")
        except:
            print("model.h5 non pré entrainé en input, non chargé")
            pass
except:
    print("aucun model.h5 chargé")
    pass

# Resultats sur plusieurs images du set de validation

In [ ]:
Test_images = val_idx[0:100]

check_generator = DataGenerator(Test_images,
                                df=mask_count_df,
                                target_df=train_df,
                                shuffle=False,
                                reshape=(HEIGHT, WIDTH),
                                augment=False,
                                n_channels=CHANNELS,
                                color_mode = COLOR_MODE,
                                n_classes=NB_CLASSES,
                                batch_size=1)

batch_pred_masks = model.predict_generator(check_generator, 
                                            workers=1,
                                            verbose=1)

In [ ]:
THRESHOLD = 0.5 # seuil à partir duquel on considère le masque prédit à 1

print(Fore.GREEN + 'Masque original')
print(Fore.YELLOW + 'Masque prédit')
print(Fore.BLUE + 'Fit du masque prédit avec le masque original')

for i in range(10):
    batch_pred_masks_thr = np.zeros(batch_pred_masks[i].shape).astype(np.uint8)
    batch_pred_masks_thr[batch_pred_masks[i] > THRESHOLD] = 1
    
    visualize_image_mask_prediction(check_generator.__getitem__(i)[0][0,:,:,:],
                                    check_generator.__getitem__(i)[1][0,:,:,:],
                                    batch_pred_masks_thr,
                                    Transparency=True)

# Calcul du dice moyen par classe de nuage

In [ ]:
# Calcul du dice moyen/class sur les images Test

dice_fish=[]
dice_flower=[]
dice_gravel=[]
dice_sugar=[]

for i in range(100):
    batch_pred_masks_thr = np.zeros(batch_pred_masks[i].shape).astype(np.uint8)
    batch_pred_masks_thr[batch_pred_masks[i] > THRESHOLD] = 1
    dice_fish.append(dice_coef_class(check_generator.__getitem__(i)[1][0,:,:,:],batch_pred_masks_thr)[0])
    dice_flower.append(dice_coef_class(check_generator.__getitem__(i)[1][0,:,:,:],batch_pred_masks_thr)[1])
    dice_gravel.append(dice_coef_class(check_generator.__getitem__(i)[1][0,:,:,:],batch_pred_masks_thr)[2])
    dice_sugar.append(dice_coef_class(check_generator.__getitem__(i)[1][0,:,:,:],batch_pred_masks_thr)[3])

print("Moyen dice fish",np.mean(dice_fish))
print("Moyen dice flower",np.mean(dice_flower))
print("Moyen dice gravel",np.mean(dice_gravel))
print("Moyen dice sugar",np.mean(dice_sugar))

In [ ]:
file_log = "./training.log"

log=pd.read_csv(file_log)
log.head()

plt.figure(figsize=(12,10))
plt.subplot(221)
plt.plot(log['loss'])
plt.plot(log['val_loss'])
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='right')

plt.subplot(222)
plt.plot(log['dice_coef'])
plt.plot(log['val_dice_coef'])
plt.title('Model dice_coef by epoch')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='right')
plt.show()
